In [2]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import database
from custom_scripts import preprocessing
from custom_scripts import preparation
from custom_scripts import modeling

## Quering the database

In [16]:
example_query =  """
                SELECT * FROM flights
                    LIMIT 10;
                """

example = database.query(example_query)
example.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-05-15,DL,DL,DL,2235,DL,N343NW,2235,11433,DTW,...,1481.0,24.0,0.0,0.0,0.0,136.0,None,NaN,NaN,None


## Fetching the the base training data, with filters applied by script

In [5]:
flights = preprocessing.get_training_flight_data()
print(flights.shape)
flights.head(1)

(1202587, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2018-01-01,9E,N931XJ,3507,10431,10397,615,724,69.0,N,1.0,164.0,49.0


## Splitting the data with size and random seed held in script

In [10]:
X = flights[['origin_airport_id','dest_airport_id']]
y = flights['arr_delay']
X_train, X_test, y_train, y_test = preparation.get_train_test_split(X, y)

## Using the modeling script to run a test, automatically saving the scores and model to local drive

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
notes = "This is an example"
modeling.run_test(X_train,X_test,y_train,y_test, model, notes)

{'r2_score': -2.1065611590653788e-06, 'mean_squared_error': 2523.296610862557, 'mean_absolute_error': 24.625005634594437, 'explained_variance_score': 6.810394955536481e-06}


LinearRegression()

## Retrieving the records of past modeling tests

In [12]:
modeling.get_records()

,model,notes,training_time,r2_score,mean_squared_error,mean_absolute_error,explained_variance_score
1.pickle,LinearRegression,This is an example,0.043895,-0.000002,2523.296611,24.625006,0.000007


## Retrieving saved model

In [13]:
saved_model = modeling.get_pickle(1)